In [ ]:
import requests
import pandas as pd
import time
import random
import os
from datetime import datetime

API_URL = "https://api.bilibili.com/x/web-interface/ranking/v2?rid=36&type=all"

HEADERS = {
    "User-Agent": "...",
    "Referer": "https://www.bilibili.com/",
    "Cookie": "YOUR_SESSDATA"
}

def fetch_ranking_data():
    """
    Fetches the top 100 videos from the specified Bilibili partition.
    """
    print(f"🚀 Connecting to Bilibili API: {API_URL}...")
    
    try:
        response = requests.get(API_URL, headers=HEADERS, timeout=10)
        
        if response.status_code == 200:
            json_data = response.json()
            
            if json_data['code'] == 0:
                video_list = json_data['data']['list']
                print(f"✅ Successfully retrieved {len(video_list)} videos.")
                return video_list
            else:
                print(f"❌ API Error Message: {json_data['message']}")
                return []
        else:
            print(f"❌ HTTP Error: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"❌ Connection Error: {e}")
        return []

def process_data(raw_list):
    """
    Cleans and extracts specific metrics needed for the analysis.
    """
    structured_data = []
    
    print("⚙️ Processing data metrics...")
    
    for video in raw_list:
        stats = video.get('stat', {})
        
        item = {
            'bvid': video.get('bvid'),
            'title': video.get('title'),
            'author': video.get('owner', {}).get('name'),
            'pub_date': datetime.fromtimestamp(video.get('pubdate')),
            'duration_sec': video.get('duration'),
            
            'views': stats.get('view'),
            'danmaku': stats.get('danmaku'), 
            'reply': stats.get('reply'),
            'favorite': stats.get('favorite'),
            'coin': stats.get('coin'),
            'share': stats.get('share'),
            'like': stats.get('like'),
            'score': video.get('score')
        }
        
        total_interactions = item['coin'] + item['favorite'] + item['like'] + item['share']
        if item['views'] > 0:
            item['engagement_rate'] = round((total_interactions / item['views']) * 100, 2)
        else:
            item['engagement_rate'] = 0
            
        structured_data.append(item)
        
    return structured_data

def main():
    raw_videos = fetch_ranking_data()
    
    if raw_videos:
        clean_data = process_data(raw_videos)
        
        df = pd.DataFrame(clean_data)
        
        timestamp = datetime.now().strftime("%Y%m%d")
        filename = f"bilibili_knowledge_rank_{timestamp}.csv"
        
        save_path = os.path.join(os.path.expanduser("~"), "Desktop", filename)
        
        try:
            df.to_csv(save_path, index=False, encoding='utf-8-sig')
            print(f"\n✅ SUCCESS! File saved to: {save_path}")
        except Exception as e:
            print(f"\n❌ Still cannot save to Desktop. Trying /tmp/ folder...")
            save_path = f"/tmp/{filename}"
            df.to_csv(save_path, index=False, encoding='utf-8-sig')
            print(f"✅ Saved to: {save_path}")

        print(f"\n📊 Data Preview (Top 5):")
        print(df[['title', 'views', 'coin', 'engagement_rate']].head())

if __name__ == "__main__":
    main()

🚀 Connecting to Bilibili API: https://api.bilibili.com/x/web-interface/ranking/v2?rid=36&type=all...
✅ Successfully retrieved 96 videos.
⚙️ Processing data metrics...

✅ SUCCESS! File saved to: /Users/huhaoquanspark/Desktop/bilibili_knowledge_rank_20260127.csv

📊 Data Preview (Top 5):
                               title    views    coin  engagement_rate
0           前8天：穿越罗布泊无人区  第9天：主角突然换人  5134086   16906            15.26
1  震撼世界的72天：巴黎公社为何爆发？又为何失败？【历史调研室71】  3359337  104426            10.81
2                 我被电视剧片头曲“模仿”了却无能为力  7007559  367587            15.69
3              为什么中国人对一道菜的最高评价是"下饭"?  3531050   12953             4.28
4                 深度|| 香火之战，车迟国的教皇实验  2359986   55540             6.21
